In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import f1_score

data = pd.read_excel('cleaned_dataset.xlsx')
y = data['Class']
transport_columns = [col for col in data.columns if 'Type_of_Transport_Used_' in col]
transport_data = data[transport_columns]
transport = transport_data.astype(int).to_numpy()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(transport, y, test_size=0.3, random_state=42)

#X_train, X_test, y_train, y_test = train_test_split(transport, y, test_size=0.3, random_state=42)

# Step 2: Initialize parameters
n_features = X_train.shape[1]
n_classes = len(np.unique(y))  # 4 obesity levels
W = np.random.randn(n_features, n_classes) * 0.01  # Small random weights
b = np.zeros((1, n_classes))  # Bias initialized to zero

# Step 3: Define helper functions
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Stability trick
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def compute_gradients(X, y_true, y_prob, y_pred):
    n_samples = X.shape[0]
    
    # Gradients for weights and bias (based on probabilities)
    dW = (1 / n_samples) * np.dot(X.T, (y_prob - np.eye(n_classes)[y_true.astype(int) - 1]))
    db = (1 / n_samples) * np.sum(y_prob - np.eye(n_classes)[y_true.astype(int) - 1], axis=0, keepdims=True)
    
    return dW, db

# Step 4: Training loop
learning_rate = 0.1
n_epochs = 1000

for epoch in range(n_epochs):
    # Forward pass
    logits = np.dot(X_train, W) + b  # Compute logits
    y_prob = softmax(logits)  # Class probabilities
    y_pred = np.sum(y_prob * np.arange(1, n_classes + 1), axis=1)  # Expected value (scalar prediction)
    
    # Compute loss
    loss = mean_squared_error(y_train, y_pred)
    
    # Backward pass
    dW, db = compute_gradients(X_train, y_train, y_prob, y_pred)
    
    # Update parameters
    W -= learning_rate * dW
    b -= learning_rate * db
    
    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Step 5: Evaluate on test set
logits_test = np.dot(X_test, W) + b
y_prob_test = softmax(logits_test)
y_pred_test = np.sum(y_prob_test * np.arange(1, n_classes + 1), axis=1)
test_loss = mean_squared_error(y_test, y_pred_test)
print(f"Test Loss: {test_loss:.4f}")
y_pred_test_rounded = np.round(y_pred_test)
f1 = f1_score(y_test, y_pred_test_rounded, average='weighted')
print(f"F1 Score: {f1:.4f}")

Epoch 0, Loss: 0.7132
Epoch 100, Loss: 0.6811
Epoch 200, Loss: 0.6777
Epoch 300, Loss: 0.6769
Epoch 400, Loss: 0.6767
Epoch 500, Loss: 0.6766
Epoch 600, Loss: 0.6766
Epoch 700, Loss: 0.6766
Epoch 800, Loss: 0.6766
Epoch 900, Loss: 0.6766
Test Loss: 0.6391
F1 Score: 0.2004
